In [1]:
import pandas as pd
import networkx as nx
#from pgmpy.models import BayesianNetwork
#from pgmpy.inference import CausalInference
import numpy as np
import random

In [2]:
def remove_direction(graph):
    # 양방향 쌍 찾기
    unique_pairs = set()
    bidirectional_pairs = [(u, v) for u, v in graph.edges() if graph.has_edge(v, u)]

    # 양방향 쌍에 대해서 속성 RR이 높은 쌍만 추출
    random_rr_pairs = []
    for u, v in bidirectional_pairs:
        sorted_pair = tuple(sorted([u, v]))  # (u, v)와 (v, u)를 정렬하여 동일한 쌍으로 처리

        if sorted_pair not in unique_pairs:  # 이미 처리된 쌍이 아닌 경우에만 처리
            # 랜덤하게 (u, v) 또는 (v, u) 중 하나를 선택
            if random.choice([True, False]):
                random_rr_pairs.append((u, v, graph[u][v]['RR']))
            else:
                random_rr_pairs.append((v, u, graph[v][u]['RR']))

            # 처리된 쌍을 집합에 추가
            unique_pairs.add(sorted_pair)

    # RR 높은 쌍과 양방향이 아닌 단방향쌍과 합치기
    all_edges = set(graph.edges()) - set(bidirectional_pairs)
    final_edges = [(u, v, graph[u][v]['RR']) for u, v in all_edges] + random_rr_pairs

    # 그래프 변수 생성하여 합친 쌍에 대하여 다시 넣기 (D1, D2, 속성 RR)
    graph2 = nx.DiGraph()
    graph2.add_edges_from([(u, v, {'RR': rr}) for u, v, rr in final_edges])

    return graph2

In [3]:
# 서브그래프에서 노드 이전 또는 이후의 엣지를 삭제하는 함수
# 여기서 삭제할 때 mediator_edges 에 있는 것들은 삭제하면 안되는 코드 작성해야함 <-------------------------------

def removeEdge(graph, subgraph, undirgraph, cycle, node, mediator_edges, flag='before'):
    node_index = cycle.index(node)

    if flag == 'before':
        # 이전 노드에서 현재 노드로 가는 엣지 삭제
        prev_node = cycle[(node_index - 1) % len(cycle)]  # 리스트 순환으로 이전 노드 선택

        if (prev_node, node) not in mediator_edges:
            # direction_subgraph와 direction_graph에서 엣지가 존재할 때만 삭제
            if subgraph.has_edge(prev_node, node):
                subgraph.remove_edge(prev_node, node)

            if graph.has_edge(prev_node, node):
                graph.remove_edge(prev_node, node)

            # undirgraph는 무방향이므로 양방향 체크 후 엣지 제거<----------------오타 수정
            if undirgraph.has_edge(node, prev_node):
                undirgraph.remove_edge(node, prev_node)
            elif undirgraph.has_edge(prev_node, node):
                undirgraph.remove_edge(prev_node, node)

    elif flag == 'after':
        # 현재 노드에서 다음 노드로 가는 엣지 삭제
        next_node = cycle[(node_index + 1) % len(cycle)]  # 리스트 순환으로 다음 노드 선택

        if (node, next_node) not in mediator_edges:
            # direction_subgraph와 direction_graph에서 엣지가 존재할 때만 삭제
            if subgraph.has_edge(node, next_node):
                subgraph.remove_edge(node, next_node)
                
            if graph.has_edge(node, next_node):
                graph.remove_edge(node, next_node)
                
            # undirgraph는 무방향이므로 양방향 체크 후 엣지 제거
            if undirgraph.has_edge(node, next_node):
                undirgraph.remove_edge(node, next_node)
            elif undirgraph.has_edge(next_node, node):
                undirgraph.remove_edge(next_node, node)

    else:
        raise ValueError("Invalid flag. Use 'before' or 'after'.")


In [4]:
# 노드의 거리를 계산하고 거리가 가장 큰 노드가 여러 개 있을 경우 랜덤으로 하나의 노드를 선택하는 함수
def get_maxnode(cycle, distances):
    max_distance = max((distances[node] for node in cycle if node in distances), default=None)
    max_nodes = [node for node in cycle if distances.get(node) == max_distance]
    return random.choice(max_nodes) if max_nodes else None

In [5]:
#distances_from_D1, distances_from_D2 의 평균 거리 계산하는 함수 <-----------------------
def average_dict_values(d1, d2):
    result = {}
    for key in d1:
        if key in d2:  # 두 dict 모두에 존재하는 키에 대해서만 처리
            result[key] = (d1[key] + d2[key]) / 2  # 값들의 평균 계산
    return result

### 여기가 메인함수

In [6]:

def remove_cycle(graph, D1, D2):
    all_paths = list(nx.all_simple_paths(graph, D1, D2, cutoff=2))

    # 고유한 노드들 필터링 (길이 5 이하인 경로들만 고려)
    unique_nodes = {node for path in all_paths for node in path}
    temp=unique_nodes.copy()
    for node in temp:
        if int(D1/1000) < int(node/1000) and node !=D2:
            unique_nodes.remove(node)
        elif int(D1/1000)> int(node/1000):
            unique_nodes.remove(node)

    mediator_edges = set() #<-----------------------------mediator edge 모음. 이것들은 나중에 삭제하면 안됨.
    for path in all_paths:
        # Convert each path (a list of nodes) to a list of edges (tuples)
        edges_in_path = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
        # Add edges to the set (to avoid duplicates)
        mediator_edges.update(edges_in_path)

    # 고유한 노드의 1-네이버 노드 포함
    neighbors = set()
    # for node in unique_nodes:
    #     neighbors.update(graph.neighbors(node))
    #     if node != D1 and node != D2: # <-----------------------mediator에 들어오는 노드는 제거 
    #         neighbors.update(graph.predecessors(node))

    # D1과 D2의 이웃 노드도 포함
    neighbors.update(graph.neighbors(D1))
    neighbors.update(graph.predecessors(D1))
    neighbors.update(graph.neighbors(D2))
    neighbors.update(graph.predecessors(D2))

    # int(D1/1000)+1 == int(D2/1000) 인 노드 삭제
    temp=neighbors.copy()
    for node in temp:
        if int(D1/1000) < int(node/1000) and node !=D2:
            neighbors.remove(node)
        elif int(D1/1000)> int(node/1000):
            neighbors.remove(node)
        
    # 모든 선택된 노드 집합 (D1, D2, 고유 노드, 이웃 노드)
    all_selected_nodes = unique_nodes.union(neighbors).union({D1, D2})

    # 서브 그래프 추출
    subgraph = graph.subgraph(all_selected_nodes).copy()
    edges_to_remove = list(graph.out_edges(D2, data=True))
    subgraph.remove_edges_from(edges_to_remove)# D2로 나가는 엣지 제거
    if subgraph.has_edge(D2, D1):
        subgraph.remove_edge(D2, D1)

    # 최종 서브그래프 생성
    subgraph = remove_direction(subgraph)

    #2. 방향이 없는 서브그래프 생성, d1, d2에 대한 거리 계산용<----------------------
    undirected_graph = nx.Graph(subgraph.copy())

    # 3. 방향 서브그래프 (이미 생성된 subgraph), 사이클 제거용<-----------------------
    directed_graph = subgraph.copy()
    
    # 사이클 제거를 위한 코드
    sccs = list(nx.strongly_connected_components(directed_graph))
    for scc in sccs:
        if len(scc) < 2:
            continue  # SCC의 크기가 1보다 큰 경우에만 사이클 존재 가능
        directed_subgraph = directed_graph.subgraph(scc).copy()
        while True:
            try:
                cycle = next(nx.simple_cycles(directed_subgraph))
                # print('Cycle:', cycle)
                
                # 거리 계산
                distances_from_D1 = nx.shortest_path_length(undirected_graph, source=D1, weight=None)
                distances_from_D2 = nx.shortest_path_length(undirected_graph, source=D2, weight=None)
                # distances_from_D1, distances_from_D2 의 평균 거리 계산 <--------------------------
                average_dict = average_dict_values(distances_from_D1, distances_from_D2)
                
                # 1. D1과 D2가 모두 포함된 경우 또는 D1 또는 D2가 포함되지 않은 경우
                if (D1 in cycle and D2 in cycle) or (D1 not in cycle and D2 not in cycle):
                    #D1과 D2의 평균 거리가 제일 먼거로 변경함 
                    max_node_d1 = get_maxnode(cycle, distances_from_D1)
                    max_node_d2 = get_maxnode(cycle, distances_from_D2)
                    max_node = get_maxnode(cycle, average_dict)
                    
                    # D1과 D2 중 더 먼 노드를 선택하여 엣지 삭제<-----------평균거리가 더 먼 노드를 삭제. 아래 부분 확인해볼 것
                    #if max_node_d1 and max_node_d2:
                    if max_node: #<----------수정됨
                        removeEdge(directed_graph, directed_subgraph, undirected_graph, cycle, max_node_d2, mediator_edges, flag='before')#<----before를 기본으로 할 것

                # 2. D1 또는 D2 중 하나만 포함된 경우
                elif D1 in cycle:
                    max_node_d1 = get_maxnode(cycle, distances_from_D1)
                    if max_node_d1:
                        removeEdge(directed_graph, directed_subgraph, undirected_graph, cycle, max_node_d1, mediator_edges, flag='after')

                elif D2 in cycle:
                    max_node_d2 = get_maxnode(cycle, distances_from_D2)
                    if max_node_d2:
                        removeEdge(directed_graph, directed_subgraph, undirected_graph, cycle, max_node_d2, mediator_edges, flag='after')

            except StopIteration:
                break
    return directed_graph

In [7]:
def export_custom_dag_to_array(subgraph):
    edges = list(subgraph.edges())
    processed_edges = set()
    result = []
    
    for u, v in edges:
        if (u, v) in processed_edges or (v, u) in processed_edges:
            continue

        if subgraph.has_edge(v, u) and subgraph.has_edge(u, v):  # 양방향 엣지 확인
            result.append(f"{u} <-> {v}")
            processed_edges.add((u, v))
            processed_edges.add((v, u))
        else:
            result.append(f"{u} -> {v}")
        processed_edges.add((u, v))
    
    return result

In [21]:
#개인이 설정할 것 ->
directory = 'G:/내 드라이브/research2024/diseasenetwork/gephi/1st round ipw/'
#directory = 'G:/내 드라이브/research2024/diseasenetwork/gephi/'
file = 'OUTPUT_IPW_M_CLEAN_STEP2.CSV'
flag =4 # 지웅 2, 경민 3, 정민 4
#<-
# 데이터 불러오기
df = pd.read_csv(directory+file)[['Source', 'Target', 'RR','Flag']]
df

,Source,Target,RR,Flag
0,1003,1006,1.397310,1
1,1003,1044,1.904844,1
2,1003,1051,1.487448,1
3,1003,1058,1.482741,1
4,1003,1059,1.486567,1
...,...,...,...,...
35250,9554,9277,1.994601,8
35251,9554,9313,2.037949,8
35252,9554,9352,1.896497,8
35253,9554,9363,2.331262,8


In [22]:
# 그래프 생성 (D1, D2, 속성=RR)
graph = nx.DiGraph()
for idx, row in df.iterrows():
    graph.add_edge(int(row['Source']), int(row['Target']), RR=row['RR'], Flag=row['Flag'])

In [10]:
print(graph.get_edge_data(6132, 6699)['Flag'])
print(graph.get_edge_data(1003, 1006)['Flag'])

0.0
15.0


In [23]:
records = []
cnt = 0 
for D1, D2 in list(graph.edges()):
    
    #flag를 확인해서 0이면 수행한 것. 여기서는 flag가 1인 것만 할 것.. 다른 프로세스에서 2,3,4로 할 것.    
    if graph.get_edge_data(D1, D2)['Flag'] != flag:
        
        continue
        
    cnt += 1

    print(D1,D2)
    g = remove_cycle(graph.copy(), D1, D2)
    
    # export_custom_dag_to_array를 통해 {D1 -> D2 ...} 형식의 배열에 담기
    edge_directions = export_custom_dag_to_array(g)
    # BayesianNetwork와 CausalInference 처리
    # bn = BayesianNetwork(g.edges())
    # infer = CausalInference(bn)
    # confounder = infer.get_minimal_adjustment_set(D1, D2)
    # confounder2 = None if not confounder else confounder
    confounder2 = None

    # 엣지 방향을 중괄호로 묶고 공백으로 구분된 문자열 생성
    edge_directions_str = "{ " + " ".join(edge_directions) + " }"

    # 기록할 데이터 추가
    records.append([D1, D2, edge_directions_str, confounder2])
    
    print('done: ', cnt)


4363 4006
done:  1
4363 4207
done:  2
4363 4231
done:  3
4363 4234
done:  4
4363 4243
done:  5
4363 4308
done:  6
4363 4310
done:  7
4363 4313
done:  8
4363 4314
done:  9
4363 4315
done:  10
4363 4318
done:  11
4363 4321
done:  12
4363 4348
done:  13
4363 4349
done:  14
4363 4350
done:  15
4363 4352
done:  16
4363 4353
done:  17
4363 4358
done:  18
4363 4362
done:  19
4363 4388
done:  20
4363 4443
done:  21
4363 4465
done:  22
4363 4468
done:  23
4363 4545
done:  24
4363 4554
done:  25
4363 5003
done:  26
4363 5006
done:  27
4363 5157
done:  28
4363 5190
done:  29
4363 5206
done:  30
4363 5207
done:  31
4363 5231
done:  32
4363 5244
done:  33
4363 5270
done:  34
4363 5308
done:  35
4363 5309
done:  36
4363 5310
done:  37
4363 5313
done:  38
4363 5314
done:  39
4363 5318
done:  40
4363 5322
done:  41
4363 5348
done:  42
4363 5350
done:  43
4363 5352
done:  44
4363 5353
done:  45
4363 5358
done:  46
4363 5362
done:  47
4363 5363
done:  48
4363 5409
done:  49
4363 5440
done:  50
4363 5443

In [24]:


# 판다스 데이터프레임 생성
df = pd.DataFrame(records, columns=["D1", "D2", "Edge Directions", "Confounder2"])

# CSV 파일로 저장
df.to_csv(directory+'output_m_'+str(flag)+'.csv', index=False)


In [25]:
len(records)

4027

In [26]:

second_column = [str(row[0])+str(row[1]) for row in records]


# 판다스 데이터프레임 생성
df = pd.DataFrame(second_column, columns=["D1D2"])

# CSV 파일로 저장
df.to_csv(directory+'temp'+str(flag)+'.csv', index=False)